In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

In [29]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir"

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir


In [30]:
df_train = pd.read_csv("master_upsampled_balanced_training_6363.csv")
df_train.shape

(19089, 238)

In [31]:
df_test = pd.read_csv("master_test.csv") 
df_test.shape

(1693, 238)

In [32]:
df_train.drop(['Unnamed: 0', 'timestamp', 'id', 'device_id', 'review_label'], axis = 1, inplace = True)
df_test.drop(['Unnamed: 0', 'timestamp', 'id', 'device_id', 'review_label'], axis = 1, inplace = True)
df_train.head()

,CO_mean,RH_mean,TP_mean,S_mean,CO_amin,RH_amin,TP_amin,S_amin,CO_amax,RH_amax,...,TP_lag,S_lag_2,CO_lag_2,RH_lag_2,TP_lag_2,S_lag_3,CO_lag_3,RH_lag_3,TP_lag_3,y
0,6669.542962,3073.072609,2497.080771,1.982348e+06,6637.550310,2714.999991,2343.999992,1.884263e+06,6715.147260,3359.406707,...,2496.965394,1.982562e+06,6669.584363,3073.404536,2496.831593,1.982644e+06,6669.527532,3073.572837,2496.657256,inconclusive
1,6425.359083,1788.479050,2252.752532,6.011676e+05,6398.646909,1348.869613,1983.194797,5.637673e+05,6455.117477,2818.000000,...,2252.466909,6.012302e+05,6425.328728,1787.900863,2252.202964,6.012556e+05,6425.313943,1787.669327,2251.960169,inconclusive
2,6541.614745,4962.899801,2183.334424,3.144986e+05,6524.075745,4123.770600,2110.733327,2.935320e+05,6557.872955,5847.132039,...,2183.474852,3.144946e+05,6541.587338,4960.535876,2183.608068,3.145053e+05,6541.579683,4959.740800,2183.734003,inconclusive
3,7368.447188,4829.144092,2476.608656,1.399103e+06,7311.656483,4194.000000,2417.999972,1.369216e+06,7383.262476,5357.295266,...,2476.663992,1.399141e+06,7368.406840,4827.103603,2476.721480,1.399165e+06,7368.389243,4826.074105,2476.779192,inconclusive
4,6421.503514,1345.895630,2624.230946,1.550181e+05,6380.776129,945.281608,2398.041217,1.443942e+05,6487.765589,1648.845669,...,2624.413211,1.550033e+05,6421.447245,1346.451360,2624.638818,1.549950e+05,6421.436463,1346.678099,2624.894428,inconclusive


In [33]:
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_train.dropna(axis = 0, inplace = True)
df_test.dropna(axis = 0, inplace = True)

In [34]:
df_train1 = df_train[df_train.y != 'inconclusive']
df_train1.reset_index(inplace = True, drop = True)
df_train1.shape

(12718, 233)

In [35]:
y_bi = df_train1['y']
x_bi = df_train1.drop(['y'], axis = 1)

In [36]:
x_bi.shape

(12718, 232)

In [37]:
scaler = StandardScaler() 
scaler.fit(x_bi)
x_scaled_bi = scaler.transform(x_bi)  

In [12]:
#x_train_f, x_test_f, y_train_f, y_test_f = train_test_split(
#     x_scaled_full, y_full, test_size=0.1, random_state = 99)
x_train_bi, x_test_bi, y_train_bi, y_test_bi = train_test_split(
     x_scaled_bi, y_bi, test_size=0.1, random_state = 99)

In [26]:
svm_params = {
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma':[.0001,.001,0.1, 1],
    'C':[1, 10, 20, 50, 100],
    'degree':[1,3,5]}

svm_model = svm.SVC(decision_function_shape='ovo')
grid_score = GridSearchCV(estimator = svm_model, param_grid = svm_params, cv = 3, n_jobs = 3)
grid_score.fit(x_scaled_bi, y_bi)

GridSearchCV(cv=3, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovo', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=3,
             param_grid={'C': [1, 10, 20, 50, 100], 'degree': [1, 3, 5],
                         'gamma': [0.0001, 0.001, 0.1, 1],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [27]:
print(grid_score.best_params_)
print(grid_score.best_score_)

{'C': 100, 'degree': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.9838912462674839


In [38]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data"
df2 = pd.read_csv("anomlies-1910-testdf.csv")
df2.head()

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data


,Unnamed: 0,CO_mean,RH_mean,TP_mean,S_mean,CO_amin,RH_amin,TP_amin,S_amin,CO_amax,...,TP_lag_2,S_lag_3,CO_lag_3,RH_lag_3,TP_lag_3,device_id,id,timestamp,review_label,y
0,0,6842.268983,4169.768626,2572.961915,3.122835e+06,6776.747392,3931.350217,2516.643748,3.083152e+06,6898.042803,...,2573.013054,3.122703e+06,6842.139992,4170.693492,2573.032038,119220402,58366,2019-10-31 23:59:36,30,dismissed
1,1,6010.315363,6140.376041,2295.123608,4.698697e+05,5785.549909,5685.999996,2210.000000,4.572246e+05,6089.000025,...,2295.257507,4.697985e+05,6011.291145,6140.744937,2295.328336,118480027,58365,2019-10-31 23:59:05,30,dismissed
2,2,9922.644315,5415.291023,2484.306578,4.123896e+06,0.000000,5355.431521,2474.578958,4.002635e+06,22481.358586,...,2484.275579,4.123716e+06,9909.539557,5415.400294,2484.259395,118400011,58364,2019-10-31 23:57:22,30,dismissed
3,3,6539.605384,5525.457523,2427.543684,8.448359e+05,6526.999925,5134.000032,2360.635487,5.486377e+05,6558.550222,...,2427.140038,8.407960e+05,6539.524178,5527.623420,2426.944147,118180002,58363,2019-10-31 23:57:16,30,dismissed
4,4,6684.992560,4420.435466,2240.490556,4.646556e+06,6660.011712,4113.675626,2120.833898,4.618726e+06,6765.847188,...,2240.463302,4.646114e+06,6684.548108,4420.934189,2240.452508,119200181,58362,2019-10-31 23:56:37,10,inconclusive


In [39]:

df2.drop(['Unnamed: 0', 'timestamp', 'id', 'device_id','review_label'], axis = 1, inplace = True)
df2 = df2.replace([np.inf, -np.inf], np.nan)
df2.dropna(axis = 0, inplace = True)
df2 = df2[df2.y != 'inconclusive']
df2.reset_index(inplace = True, drop = True)
y_bi1 = df2['y']
x_bi1 = df2.drop(['y'], axis = 1)
scaler = StandardScaler() 
scaler.fit(x_bi1)
x_scaled_bi1 = scaler.transform(x_bi1) 

In [40]:
x_bi.shape

(12718, 232)

In [41]:
len((y_bi1[y_bi1 == 'dismissed']))/len(y_bi1)


0.9050169612569184

In [42]:
#was .76
svm_mod = svm.SVC(decision_function_shape='ovo', C = 100,degree = 1, gamma = 0.001, kernel = 'rbf').fit(x_train_bi, y_train_bi)
svm_pred = svm_mod.predict(x_scaled_bi1)
accuracy_score(y_bi1, svm_pred)

0.7751294411712194

In [43]:
print(classification_report(y_bi1, svm_pred))

               precision    recall  f1-score   support

clear_smoking       0.30      0.99      0.46      1064
    dismissed       1.00      0.75      0.86     10138

     accuracy                           0.78     11202
    macro avg       0.65      0.87      0.66     11202
 weighted avg       0.93      0.78      0.82     11202



In [26]:
confusion_matrix(y_test_f, svm_pred, labels = ['clear_smoking', 'inconclusive','dismissed'])

array([[244,   0,   7],
       [ 92,   0, 197],
       [  8,   0, 287]], dtype=int64)

In [34]:
confusion_matrix(y_bi1, svm_pred, labels = ['clear_smoking', 'inconclusive','dismissed'])

array([[1057,    0,    7],
       [   0,    0,    0],
       [2736,    0, 7859]], dtype=int64)

In [29]:
series_pred = pd.Series(svm_pred)
series_pred.index = y_test.index
series_pred.head()

74       inconclusive
413      inconclusive
1179    clear_smoking
6578    clear_smoking
270      inconclusive
dtype: object

In [36]:
df99 = pd.concat([y_test, review_label, series_pred], axis = 1, join = 'inner')
df99.rename(columns={0: 'pred'}, inplace = True)
df99.reset_index(drop = True, inplace = True)
df99.head()

,y,review_label,pred
0,inconclusive,10,inconclusive
1,dismissed,30,inconclusive
2,clear_smoking,2,clear_smoking
3,clear_smoking,2,clear_smoking
4,inconclusive,10,inconclusive


In [38]:
flag = []
for i,j in enumerate(df99['y']):
    if j == df99.loc[i,'pred']:
        flag.append('True')
    else:
        flag.append('False')
df99['correct?'] = flag
df99.head()

,y,review_label,pred,correct?
0,inconclusive,10,inconclusive,True
1,dismissed,30,inconclusive,False
2,clear_smoking,2,clear_smoking,True
3,clear_smoking,2,clear_smoking,True
4,inconclusive,10,inconclusive,True


In [50]:
df99[df99['correct?'] == 'True'].groupby('review_label')['review_label'].count()

review_label
1      20
2     175
3      23
10    177
11      1
13      5
20      9
30    193
33      1
36      8
50     22
Name: review_label, dtype: int64

In [49]:
df99[df99['correct?'] == 'False'].groupby('review_label')['review_label'].count()

review_label
1      5
2     26
3      2
10    93
11     1
13    12
20    13
30    39
31     1
33     4
34     2
35     3
Name: review_label, dtype: int64

In [47]:
new_df.groupby('review_label')['review_label'].count()

review_label
1      20
2     175
3      23
10    177
11      1
13      5
20      9
30    193
33      1
36      8
50     22
Name: review_label, dtype: int64